<a href="https://colab.research.google.com/github/RajuGuguloth/DL-Assignment-2/blob/main/DL_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Raju Gugoloth DA6401 Assignment 2
"""

# Only allowed and necessary libraries
!pip install -q wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import os
import random
import numpy as np
from typing import List, Tuple


In [1]:
# Only allowed and necessary libraries
!pip install -q wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import os
import random
import numpy as np
from typing import List, Tuple


In [2]:
class NeuralModel(nn.Module):
    def __init__(self,
                 input_shape: Tuple[int, int, int] = (3, 64, 64),
                 num_classes: int = 10,
                 conv_configs: List[dict] = None,
                 dense_units: int = 128,
                 activation: nn.Module = nn.ReLU):
        super(NeuralModel, self).__init__()
        self.features = nn.Sequential()

        in_channels = input_shape[0]
        h, w = input_shape[1], input_shape[2]

        # Convolutional Blocks
        for i, config in enumerate(conv_configs):
            out_channels = config['out_channels']
            kernel_size = config['kernel_size']
            act = config['activation']

            self.features.add_module(f"conv{i}", nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1))
            self.features.add_module(f"act{i}", act())
            self.features.add_module(f"pool{i}", nn.MaxPool2d(kernel_size=2))

            in_channels = out_channels
            h //= 2
            w //= 2

        self.flattened_size = in_channels * h * w

        self.classifier = nn.Sequential(
            nn.Linear(self.flattened_size, dense_units),
            activation(),
            nn.Linear(dense_units, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [3]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total trainable parameters: {total}")
    return total


In [5]:
# Define your conv layer configurations
conv_configs = [
    {"out_channels": 32, "kernel_size": 3, "activation": nn.ReLU},
    {"out_channels": 64, "kernel_size": 3, "activation": nn.ReLU},
    {"out_channels": 128, "kernel_size": 3, "activation": nn.ReLU},
]

# Define the device
# If a GPU is available, it will use the GPU. Otherwise, it will use the CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Build model instance
model = NeuralModel(
    input_shape=(3, 64, 64),  # assuming iNaturalist images are resized to 64x64
    num_classes=10,
    conv_configs=conv_configs,
    dense_units=128,
    activation=nn.ReLU
).to(device)

# Print parameter count
count_parameters(model)

# Print model structure
print(model)

Total trainable parameters: 1143242
NeuralModel(
  (features): Sequential(
    (conv0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act0): ReLU()
    (pool0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act2): ReLU()
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=8192, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')